# 推論用コード
Hugging Faceにアップロードしたモデルを用いてELYZA-tasks-100-TVの出力を得るためのコードです。  

In [1]:
# llama-cpp-pythonをCUDAサポート付きでインストール
!pip uninstall -y llama-cpp-python
!CMAKE_ARGS="-DGGML_CUDA=on -DCUDA_PATH=/usr/local/cuda-12.2 -DCUDAToolkit_ROOT=/usr/local/cuda-12.2 -DCUDAToolkit_INCLUDE_DIR=/usr/local/cuda-12/include -DCUDAToolkit_LIBRARY_DIR=/usr/local/cuda-12.2/lib64 -DCMAKE_CUDA_COMPILER:PATH=/usr/local/cuda/bin/nvcc" FORCE_CMAKE=1 pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122 --upgrade --force-reinstall --no-cache-dir
!pip install -U ipywidgets
!pip install transformers==4.46.3
!pip install -U bitsandbytes
!pip install -U accelerate
!pip install -U datasets
!pip install -U peft==0.13.2
!pip install janome

In [16]:
import json
import logging
from llama_cpp import Llama
from tqdm import tqdm
from typing import Dict, Any
import numpy as np
import logging
from dataclasses import dataclass
from enum import Enum
import re
from janome.tokenizer import Tokenizer

In [ ]:
# ロギングの設定
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [4]:
# モデル設定
# GGUFモデルのパス
MODEL_PATH = "./llm-jp-3-13b-q8_0.gguf"
# LoRAアダプタのパス
lora_path = "llm-jp-3-13b-SFT-LoRA.gguf"
FILE_NAME = "llm-jp-3-13b-SFT-LoRA.jsonl"

In [ ]:
# タスク種別判定用のキーワード
KEYWORDS = {
    '説明': [
        ('説明して', 0.9), ('解説して', 0.9), ('とは', 0.7), ('どういうこと', 0.8),
        ('なぜ', 0.8), ('どうして', 0.8), ('理由', 0.7), ('方法', 0.5), ('手順', 0.5),
        ('仕組み', 0.8), ('プロセス', 0.7), ('メカニズム', 0.8), ('定義', 0.9), ('概要', 0.7),
        ('言い換えて', 0.8)
    ],
    '物語': [
        ('物語', 1.0), ('ストーリー', 0.9), ('話を作って', 0.9), ('創作', 0.8),
        ('昔話', 0.8), ('童話', 0.8), ('小説', 0.9), ('脚本', 0.9), ('ファンタジー', 0.7),
        ('冒険', 0.7), ('恋愛', 0.7), ('ミステリー', 0.7), ('SF', 0.7)
    ],
    '推論・分析': [
        ('推論', 1.0), ('考察', 0.9), ('分析', 0.9), ('結論', 0.8), ('予測', 0.8),
        ('考えられる', 0.7), ('推定', 0.7), ('推測', 0.7), ('判断', 0.7),
        ('なぜなら', 0.9), ('したがって', 0.8), ('結果', 0.7), ('原因', 0.8), ('要因', 0.8),
        ('論理的に', 0.8), ('根拠', 0.7), ('情報', 0.7)
    ],
    '指示・要請': [
        ('してほしい', 0.9), ('してください', 0.9), ('必要', 0.8), ('要請', 0.8),
        ('依頼', 0.8), ('命令', 0.9), ('べき', 0.7), ('課題', 0.6), ('方法', 0.6),
        ('手順', 0.6), ('やり方', 0.6)
    ],
    '抽出・選択': [
        ('抽出', 1.0), ('抜き出し', 0.9), ('選択', 0.9), ('選んで', 0.9), ('分類', 0.7),
        ('リストアップ', 0.8), ('該当する', 0.8), ('当てはまる', 0.8),
        ('どれ', 0.9), ('どれか', 0.9), ('選定', 0.8), ('特定', 0.8)
    ],
    '事実確認': [
        ('いつ', 0.9), ('どこで', 0.9), ('誰が', 0.9), ('何を', 0.9), ('いくつ', 0.9),
        ('本当', 0.8), ('正しい', 0.8), ('事実', 0.8), ('確認', 0.7), ('情報', 0.7),
        ('データ', 0.6), ('証拠', 0.7), ('信頼できる', 0.8)
    ],
    '意見・推測': [
        ('どう思う', 0.9), ('意見', 0.9), ('考え', 0.8), ('推測', 0.7), ('予想', 0.7),
        ('可能性', 0.7), ('見解', 0.8), ('立場', 0.8), ('賛成', 0.7), ('反対', 0.7),
        ('評価', 0.7)
    ],
    '評価・レビュー': [
        ('評価', 0.9), ('レビュー', 0.9), ('感想', 0.8), ('批評', 0.8), ('良い点', 0.7),
        ('悪い点', 0.7), ('メリット', 0.7), ('デメリット', 0.7), ('改善点', 0.8),
        ('5段階評価', 0.7), ('星', 0.6), ('点数', 0.6)
    ],
    '変換・言い換え': [
        ('変換', 0.9), ('言い換え', 0.9), ('言い換えて', 0.8), ('別の表現', 0.8), ('置き換え', 0.8),
        ('リフレーズ', 0.7), ('パラフレーズ', 0.7), ('同義語', 0.7), ('類義語', 0.7)
    ],
    '比較': [
        ('比較', 1.0), ('比べて', 0.9), ('違い', 0.9), ('差', 0.8), ('差異', 0.8),
        ('共通点', 0.9), ('類似点', 0.9), ('対比', 0.8), ('対照', 0.8)
    ],
    '分類・カテゴライズ': [
        ('分類', 1.0), ('カテゴライズ', 0.9), ('グループ分け', 0.9), ('仕分け', 0.8),
        ('カテゴリー', 0.8), ('ジャンル', 0.8), ('種類', 0.8), ('タイプ', 0.8),
        ('属性', 0.7), ('要素', 0.6), ('項目', 0.6)
    ],
    'ロールプレイ': [
        ('役割', 0.9), ('役', 0.8), ('ロールプレイ', 1.0), ('になりきって', 0.9),
        ('ふりをして', 0.8), ('演じて', 0.9), ('キャラクター', 0.8), ('設定', 0.7),
        ('シチュエーション', 0.7),('あなたは', 0.8)
    ],
    '生成・作成': [
        ('生成', 0.9), ('作成', 0.9), ('作って', 0.9), ('書いて', 0.8), ('創作', 0.8),
        ('作り出して', 0.8), ('生み出す', 0.8), ('考案', 0.8), ('デザイン', 0.7),
        ('設計', 0.7), ('構築', 0.7), ('構成', 0.7), ('新しい', 0.7),('台本を書いて', 0.9)
    ],
    'ガイド・アドバイス': [
        ('ガイド', 0.9), ('案内', 0.8), ('紹介', 0.7), ('説明', 0.6), ('指導', 0.7),
        ('アドバイス', 0.9), ('提案', 0.8), ('ヒント', 0.7), ('コツ', 0.7),
        ('手順', 0.6), ('方法', 0.6), ('進め方', 0.7), ('使い方', 0.7)
    ],
    'その他': [
        ('その他', 0.0)
    ]
}

In [ ]:
# タスク種別に応じたパラメータ（max_tokensはデフォルト値を設定）
PARAMS = {
    '説明': {'max_tokens': 2048, 'temperature': 0.4, 'top_p': 0.85, 'top_k': 40, 'frequency_penalty': 0.1},
    '物語': {'max_tokens': 4096, 'temperature': 0.7, 'top_p': 0.85, 'top_k': 40, 'frequency_penalty': 0.2},
    '推論・分析': {'max_tokens': 2048, 'temperature': 0.2, 'top_p': 0.8, 'top_k': 40, 'frequency_penalty': 0.1},
    '指示・要請': {'max_tokens': 1024, 'temperature': 0.1, 'top_p': 0.7, 'top_k': 40, 'frequency_penalty': 0.0},
    '抽出・選択': {'max_tokens': 768, 'temperature': 0.0, 'top_p': 0.6, 'top_k': 40, 'frequency_penalty': 0.0},
    '事実確認': {'max_tokens': 768, 'temperature': 0.0, 'top_p': 0.6, 'top_k': 40, 'frequency_penalty': 0.0},
    '意見・推測': {'max_tokens': 1024, 'temperature': 0.5, 'top_p': 0.8, 'top_k': 40, 'frequency_penalty': 0.1},
    '評価・レビュー': {'max_tokens': 2048, 'temperature': 0.3, 'top_p': 0.8, 'top_k': 40, 'frequency_penalty': 0.1},
    '変換・言い換え': {'max_tokens': 1024, 'temperature': 0.2, 'top_p': 0.7, 'top_k': 40, 'frequency_penalty': 0.1},
    '要約': {'max_tokens': 768, 'temperature': 0.2, 'top_p': 0.7, 'top_k': 40, 'frequency_penalty': 0.1},
    '翻訳': {'max_tokens': 2048, 'temperature': 0.3, 'top_p': 0.8, 'top_k': 40, 'frequency_penalty': 0.0},
    '感情分析': {'max_tokens': 768, 'temperature': 0.2, 'top_p': 0.7, 'top_k': 40, 'frequency_penalty': 0.1},
    '対話': {'max_tokens': 2048, 'temperature': 0.6, 'top_p': 0.8, 'top_k': 40, 'frequency_penalty': 0.1},
    'ソート・並び替え': {'max_tokens': 1024, 'temperature': 0.0, 'top_p': 0.6, 'top_k': 40, 'frequency_penalty': 0.0},
    '比較': {'max_tokens': 2048, 'temperature': 0.4, 'top_p': 0.8, 'top_k': 40, 'frequency_penalty': 0.1},
    '分類・カテゴライズ': {'max_tokens': 2048, 'temperature': 0.2, 'top_p': 0.7, 'top_k': 40, 'frequency_penalty': 0.1},
    'ロールプレイ': {'max_tokens': 2048, 'temperature': 0.6, 'top_p': 0.8, 'top_k': 40, 'frequency_penalty': 0.2},
    '生成・作成': {'max_tokens': 4096, 'temperature': 0.7, 'top_p': 0.85, 'top_k': 40, 'frequency_penalty': 0.2},
    'ガイド・アドバイス': {'max_tokens': 2048, 'temperature': 0.4, 'top_p': 0.85, 'top_k': 40, 'frequency_penalty': 0.1},
    'その他': {'max_tokens': 1024, 'temperature': 0.2, 'top_p': 0.8, 'top_k': 40, 'frequency_penalty': 0.1}
}

In [ ]:
# タスク種別に応じた指示
INSTRUCTIONS = {
    '説明': "中学生にも理解できるように、専門用語を避け、具体例や理由を示しながら詳しく説明してください。最初に結論を簡潔に述べてから、その結論に至った理由や詳細な説明を続けてください。",
    '物語': "起承転結を意識し、登場人物の心情や情景が伝わるように、具体的な描写を心がけたオリジナルの物語を作成してください。物語の最後には、教訓やメッセージを簡潔に記述してください。",
    '推論・分析': "論理的に推論・分析を行い、結論を導き出してください。推論の過程をステップバイステップで説明し、結論だけでなく、その結論に至った理由も明確に示してください。",
    '指示・要請': "具体的かつ明確な手順を示して、適切に対応してください。必要な情報が不足している場合は、追加で必要な情報を質問してください。回答は箇条書きで記述してください。",
    '抽出・選択': "指定された情報や条件に合致するものを抽出・選択してください。選択肢がある場合は、最適なものを1つ選び、その理由を明確に説明してください。回答は箇条書きで記述してください。",
    '事実確認': "信頼できる情報源に基づいた正確な情報を提供してください。可能であれば情報源を明示してください。回答は明確に記述してください。",
    '意見・推測': "あなた自身の意見や推測を述べてください。意見の根拠や、推測に至った理由を明確に説明してください。異なる意見や推測がある場合は、それらにも言及し、多角的な視点を提供してください。",
    '評価・レビュー': "評価基準に基づいて評価・レビューしてください。良い点と悪い点を具体的に挙げ、改善点があれば提案してください。評価は客観的かつ具体的であることを心がけ、5段階評価などの指標を用いることも検討してください。レビューの最後には総合評価を記述してください。",
    '変換・言い換え': "指定されたスタイルや表現に変換・言い換えてください。意味が変わらないように注意し、変換・言い換えの前後で内容に齟齬がないことを確認してください。可能であれば、変換・言い換えの例を複数提示してください。",
    '要約': "指定された文字数以内で要約してください。文章の要点や趣旨を簡潔にまとめ、重要なキーワードを必ず含めてください。要約は、元の文章を読んだことがない人にも内容が理解できるように記述してください。",
    '翻訳': "指定された言語に翻訳してください。直訳ではなく、翻訳先の言語として自然な表現になるように心がけ、文化的背景も考慮してください。また、原文のスタイルやトーンを維持するようにしてください。",
    '感情分析': "書き手の感情や心情を分析してください。ポジティブ、ネガティブ、ニュートラルのいずれかに分類し、その理由を具体的に説明してください。感情を表現する言葉やフレーズに注目し、文脈も考慮して分析してください。",
    '対話': "設定やキャラクターに基づいて、自然な対話を生成してください。キャラクターの口調や性格を反映させ、会話の流れが自然になるように心がけてください。対話の目的やゴールを意識し、会話の展開を工夫してください。",
    'ソート・並び替え': "指定された基準に従って並び替えてください。並び替えのルールを明確に示し、並び替えの結果を明確に提示してください。",
    '比較': "それぞれの特徴や違い、共通点を、表形式を用いて分かりやすく説明してください。比較の観点を明確に示し、それぞれの項目について、具体的な例を挙げて説明してください。",
    '分類・カテゴライズ': "指定されたカテゴリーや基準に基づいて分類・カテゴライズしてください。分類のルールを明確に示し、それぞれの項目がどのカテゴリーに属するのか、その理由を説明してください。分類の結果は、表形式でまとめることも検討してください。",
    'ロールプレイ': "役割やキャラクターになりきって、与えられたシチュエーションに対して適切に応答してください。キャラクターの性格や口調、背景情報を反映させ、シチュエーションに即した自然な応答を心がけてください。応答は、キャラクターの視点から、一人称で記述してください。",
    '生成・作成': "指示やテーマに基づいて、オリジナリティのある新しいコンテンツを生成・作成してください。具体的かつ詳細な内容を心がけ、必要に応じて、例や説明を加えてください。生成・作成したコンテンツは、明確な構成で、読みやすく記述してください。",
    'ガイド・アドバイス': "状況や課題に対して、具体的かつ実践的なガイドやアドバイスを提供してください。手順や方法を示す必要がある場合は、ステップバイステップで、分かりやすく説明してください。ガイドやアドバイスは、対象者の立場や状況を考慮し、実現可能なものを提示してください。",
    'その他': "内容を確認し、適切な対応をしてください。指示が不明確な場合は、具体的にどの部分が不明確なのかを質問し、指示の明確化を求めてください。回答は、簡潔かつ明確に記述してください。"
}

In [ ]:
FEW_SHOT_EXAMPLES = {
    "説明": [
        {
            "input": "あの、娘がやっているあのキ、チックトック？チックトッカー？っていうのは何なんですか？",
            "output": "娘さんがやっていらっしゃるのは、おそらくTikTok (ティックトック) というSNSのことですね。\n\nTikTokは、15秒から10分程度の短い動画を投稿・視聴できるアプリです。特に10代から20代の若者に人気があります。音楽に合わせて踊ったり、リップシンク（口パク）したりする動画が多いのが特徴です。ティックトッカーというのは、TikTokに動画を投稿している人たちのことを指します。\n\nTikTokが若者に人気がある理由は、誰でも簡単に動画を作成・編集できること、そして短い時間で楽しめることが挙げられます。また、TikTok独自のアルゴリズムにより、ユーザーの好みに合った動画が次々におすすめされるため、ついつい長時間見てしまうという人も多いようです。"
        },
        {
            "input": "ネットスラングでは、似た音を持つ別の短い表現に表されることがあります。例えばサーバーは鯖（さば）と表現されます。\n\nこれらを踏まえて、「消防車」はネットスラングでどのように表現されるか、説明をしてください。",
            "output": "結論から言うと、「消防車」はネットスラングで「FD」と表現されることがあります。\n\nこれは、「Fire Department」（消防署）の頭文字を取ったものです。消防車は消防署に所属しているため、このように呼ばれるようになりました。また、「FD」は、消防車だけでなく、消防署や消防士を指す場合にも使われます。\n\nちなみに、救急車は「Ambulance」の頭文字を取って「AM」と呼ばれることがあります。"
        },
        {
            "input": "「エモい」という言葉を最近よく聞きます。どういう意味か説明していただけませんか？",
            "output": "「エモい」とは、感情が揺さぶられるような、懐かしさや切なさを感じるときに使われる若者言葉です。英語の「emotional」（エモーショナル）を略した言葉で、「感情的な」「情緒的な」という意味があります。\n\n例えば、懐かしい曲を聴いたときや、昔の写真を見たとき、美しい景色を見たときなどに、「エモい」と感じることが多いです。また、感動的な映画や小説などに対しても使われます。\n\n「エモい」は、単に「感動した」というよりも、より深く、複雑な感情を表す言葉として使われます。心が揺さぶられるような、何とも言えない気持ちになったときに、「エモい」という言葉がぴったりくるでしょう。"
        }
    ],
    "物語": [
        {
            "input": "クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。",
            "output": "森に住むクマのテディは、毎日同じような生活に飽き飽きしていました。「もっと違う世界を見てみたい！」そう思ったテディは、ある日、海を目指して旅に出ました。\n\n森を抜け、草原を抜け、長い道のりを歩いて、テディはついに海辺にたどり着きました。目の前に広がる青い海、波の音、潮の香り、すべてがテディにとって初めての経験でした。「すごい！これが海か！」テディは興奮して、砂浜を走り回りました。\n\nふと、テディは沖の方に何かがいることに気づきました。目を凝らしてみると、それはアザラシの群れでした。テディはアザラシたちに近づいて、「こんにちは！僕はテディ。君たちは？」と話しかけました。アザラシたちは最初、大きなクマに驚きましたが、テディが優しそうなことに気づき、すぐに仲良くなりました。「僕はポール」「私はサリー」「僕はジョージ」アザラシたちは口々に自己紹介をしてくれました。\n\nテディはアザラシたちと毎日一緒に遊びました。泳ぎ方を教えてもらったり、魚の捕り方を教えてもらったり、一緒に砂浜で昼寝をしたりしました。テディにとって、海辺での生活は、毎日が新しい発見と喜びに満ちていました。\n\nしかし、楽しい時間はあっという間に過ぎていきます。ある日、テディは森に住む家族のことを思い出しました。「お父さん、お母さん、元気かな…」テディは、家族に会いたくなりました。\n\nテディはアザラシたちに別れを告げることにしました。「みんな、今までありがとう。僕は森に帰るよ。」テディは涙をこらえながら言いました。「テディ、行かないで！」サリーが悲しそうな声で言いました。「また会えるよね？」ポールが不安そうに尋ねました。「もちろん！また必ず会いに来るよ！」テディは力強く答えました。\n\nテディはアザラシたちに見送られながら、森へと帰っていきました。海辺での楽しい思い出を胸に、テディは一歩一歩、家路につきました。テディは、海で友達ができたこと、そして、やっぱり自分の家が一番落ち着く場所だということを、家族に話したくてたまりませんでした。\n\n【教訓】新しい世界に飛び込むことは、素晴らしい経験をもたらしてくれます。しかし、どんなに楽しい場所でも、自分の帰るべき場所、大切な人たちがいる場所は、かけがえのないものです。"
        },
        {
            "input": "次の物語の展開を予想してみましょう。\n\nある日、主人公のもとに不思議な手紙が届きました。手紙には「今夜の満月に、森の奥深くにある洞窟に来てください。あなたを待っています。」と書かれていました。主人公はその手紙に従い、夜中に洞窟の入り口にたどり着きました。中に入ると、謎めいた人物と出会い……。",
            "output": "主人公が洞窟の中に入ると、そこにはローブをまとった謎めいた人物が立っていました。その人物は静かに言いました。「よく来たな。お前が来るのを待っていたぞ。」\n\n主人公は恐る恐る尋ねました。「あなたは誰ですか？なぜ私をここに呼んだのですか？」謎めいた人物は答えました。「私はこの森の守護者だ。お前を呼んだのは、お前に試練を与えるためだ。」\n\n「試練？」主人公は戸惑いました。「そうだ。この洞窟の奥には、古の秘宝が眠っている。その秘宝を手に入れることができれば、お前は強大な力を得ることができるだろう。しかし、秘宝を手に入れるためには、三つの試練を乗り越えなければならない。」\n\n謎めいた人物は、洞窟の奥へと続く道を指差しました。「さあ、行くがよい。最初の試練は、勇気を示すことだ。」主人公は意を決して、洞窟の奥へと進んでいきました。洞窟の中は暗く、不気味な雰囲気が漂っていました。どこからか、獣のうなり声のようなものも聞こえてきます。\n\nしばらく進むと、主人公の前に大きな穴が現れました。穴の底は見えず、向こう岸に渡るには、細い一本橋を渡るしかありません。しかも、橋の下には、巨大なクモが巣を張って待ち構えていました。「これが最初の試練か…」主人公は恐怖に震えながらも、勇気を振り絞って、橋を渡り始めました。\n\n【この後の展開の例】\n\n*   主人公は、クモの巣にかかってしまうが、なんとか脱出し、向こう岸にたどり着く。\n*   主人公は、クモと戦い、見事勝利する。\n*   主人公は、クモを説得し、橋を渡らせてもらう。\n\n果たして、主人公は最初の試練を乗り越えることができるのでしょうか？そして、残りの二つの試練とは？主人公は無事に秘宝を手に入れることができるのでしょうか？"
        },
        {
            "input": "「桃太郎」をホラー風にアレンジしてください。",
            "output": "老婆が川で洗濯をしていると、上流から大きな桃がどんぶらこ、どんぶらこと流れてきました。よく見ると、その桃には人の顔のような模様が浮かび上がっており、不気味な笑みを浮かべているように見えます。老婆は恐ろしくなり、家に帰ろうとしましたが、背後から「待って…おばあさん…」と嗄れた声が聞こえてきました。振り返ると、先ほどの桃が、まるで生きているかのように、こちらに向かって転がってきます。「ひぃ！」老婆は悲鳴を上げ、一目散に逃げ出しました。\n\n老婆は家に逃げ込み、戸を固く閉ざしました。しかし、桃は戸をドンドンと叩き始めます。「おばあさん…開けておくれ…」桃の声は、老婆の耳にこびりついて離れません。やがて、桃は戸を破り、家の中に転がり込んできました。「おばあさん…一緒に暮らそう…」老婆は恐怖のあまり、気を失ってしまいました。\n\n老婆が目を覚ますと、桃は割れており、中から血まみれの赤ん坊が出てきました。赤ん坊は老婆を見つめ、「おばあちゃん…」と不気味な笑顔を浮かべました。老婆は悲鳴を上げようとしましたが、声が出ません。赤ん坊は老婆に這い寄り、その小さな手で老婆の首を絞め始めました。「おばあちゃん…ずっと一緒だよ…」老婆は、赤ん坊の冷たい目を見つめながら、意識を失っていきました。\n\n数年後、老婆の家には、老婆と、老婆に育てられた「桃太郎」と呼ばれる少年が暮らしていました。桃太郎は、老婆に「鬼ヶ島に行って、鬼を退治してくる」と言い、家を出ていきました。老婆は、桃太郎の後ろ姿を見つめながら、不気味な笑みを浮かべました。「鬼ヶ島には、美味しいご馳走がたくさんあるよ…」老婆は、そう呟きました。\n\n【この後の展開の例】\n\n*   桃太郎は、鬼ヶ島で鬼に食べられてしまう。\n*   桃太郎は、鬼を退治するが、その代わりに自分が鬼になってしまう。\n*   桃太郎は、鬼ヶ島で宝物を見つけるが、その宝物には呪いがかけられていた。\n\n果たして、桃太郎の運命は？そして、老婆の真の目的とは？"
        }
    ],
    "推論・分析": [
        {
            "input": "以下の文章について、筆者がどのような意図でこの文章を書いたか、あなたならどのように解釈しますか。\n\n彼はバブル期に大金持ちになり豊かな生活に日夜明け暮れたが、バブル崩壊後にほとんどの資産と職を失った。彼女に出会ったのは、そんな人生が真っ暗になってしまっていたときだった。彼は現在アルバイトで貧しいながらも生計を立てているが、それでも彼は今が一番豊かだと言っている。",
            "output": "筆者はこの文章を通じて、「本当の豊かさとは何か」を問いかけていると解釈できます。バブル期に大金持ちだった彼は、物質的には恵まれていましたが、精神的には満たされていなかったのかもしれません。バブル崩壊後、資産と職を失い、人生が真っ暗になったと感じた彼が出会ったのは、「彼女」という存在でした。彼は現在、アルバイトで生計を立てるという、バブル期とは対照的な生活を送っていますが、「今が一番豊かだ」と言っています。この言葉から、彼にとっての「豊かさ」とは、物質的なものではなく、精神的なものであることがわかります。つまり、彼女との出会いによって、彼は心の豊かさを手に入れたのです。筆者は、この物語を通じて、読者に「本当の豊かさとは、お金や物ではなく、心の豊かさである」というメッセージを伝えようとしているのではないでしょうか。"
        },
        {
            "input": "あなたは古代エジプトの貴族です。冠位によって割り振られた土地について税を納めようとしています。しかし、ある土地についてどの貴族が所有しているかを知りません。以下はその土地の形状と特徴です。\n- 南北に細長い\n- ナイル川から東に離れている\n- 西側は岩山になっている\nこの土地は誰が所有しているのでしょうか？",
            "output": "ご質問ありがとうございます。古代エジプトの貴族として、ご質問の土地の所有者について推測いたします。\n\nまず、土地の形状が「南北に細長い」とのことですが、これは古代エジプトの土地の割り当て方と一致します。ナイル川沿いの肥沃な土地は、川の流れに沿って南北に細長く区画されることが一般的でした。これは、すべての土地がナイル川の氾濫による恩恵を受けられるようにするための工夫です。\n\n次に、「ナイル川から東に離れている」とのことですが、これは土地の価値がそれほど高くないことを示唆しています。ナイル川に近い土地ほど肥沃で価値が高く、有力な貴族に割り当てられる傾向がありました。一方、川から離れた土地は乾燥しており、価値が低いと見なされていました。\n\n最後に、「西側は岩山になっている」とのことですが、これは土地の利用に制限があることを意味します。岩山は農耕に適さないため、この土地は主に牧畜や採石などに利用されていたと考えられます。\n\n以上の情報から、この土地は、それほど有力ではない貴族、あるいは神殿などに割り当てられた土地である可能性が高いと推測されます。具体的に誰が所有しているかを特定するには、当時の土地台帳などの記録を調べる必要がありますが、そのような記録が現存しているかどうかは不明です。\n\nなお、古代エジプトでは、土地の所有権はファラオに帰属し、貴族はファラオから土地を貸し与えられるという形をとっていました。そのため、税の納付先は、土地を所有する貴族ではなく、ファラオ（またはその代理人）となります。"
        },
        {
            "input": "次の文は1つの物語の文の順番をシャッフルしたものです。適切な順に並び替えてください。\n\n1. 「果物だけではなく、人生も同じです。協力しなくては、全てをこなすことはできません。」\n2. 老人は、広い畑で色々な果物を育てていました。\n3. ある日、孫が老人に『こんなに広い畑で、全ての果物を一人で育てるのは大変そうですね』と尋ねました。\n4. 昔、あるところに老人と孫がいました。\n5. 老人は微笑みながら次のように答えました。\n",
            "output": "ご質問ありがとうございます。物語の順番を並び替えてみましょう。\n\nまず、物語の始まりとして最も適切なのは、登場人物と背景を紹介する「4. 昔、あるところに老人と孫がいました。」です。次に、老人がどのような人物かを説明する「2. 老人は、広い畑で色々な果物を育てていました。」が続くと自然です。\n\nそして、孫のセリフである「3. ある日、孫が老人に『こんなに広い畑で、全ての果物を一人で育てるのは大変そうですね』と尋ねました。」が続き、それに対する老人の返答「5. 老人は微笑みながら次のように答えました。」となります。最後に、老人のセリフの内容である「1. 「果物だけではなく、人生も同じです。協力しなくては、全てをこなすことはできません。」」で締めくくると、物語として自然な流れになります。\n\nしたがって、適切な順番は **4, 2, 3, 5, 1** です。"
        }
    ],
    "指示・要請": [
        {
            "input": "youtubeの台本を書くのを手伝ってください。\n\nタローとコタローという2人のキャラクターの対話ベースで進行します。\n\nタローは物知りで、コタローの疑問に答えてくれます。\n\n今回の動画は、映画スラムダンクを視聴者に薦める動画です。以下のポイントを押さえた脚本にしてください。\n\n- まるでスポーツの試合を見ているかのように迫力のあるバスケのシーン\n- 漫画の原作を読んだことのない人でも楽しめる、映画として完結しているストーリー\n- 来月に公開が終了する\n- 10行~15行程度の対話",
            "output": "わかりました。次のような台本はいかがでしょうか？\n\n```\nコタロー: ねえタロー、最近何か面白い映画見た？\nタロー: 最近見た中では、スラムダンクの映画がすごく面白かったよ。\nコタロー: スラムダンクって、あのバスケの漫画の？映画やってたんだ。\nタロー: そうなんだ。バスケのシーンが本当に迫力があって、まるで本物の試合を見ているみたいだったよ。\nコタロー: へえ、それはすごいね。でも、僕、原作の漫画を読んだことないんだけど、楽しめるかな？\nタロー: 大丈夫！映画でストーリーが完結しているから、原作を知らなくても十分楽しめるよ。\nコタロー: そうなんだ。それはいいね。いつまで上映してるの？\nタロー: 来月で公開終了みたいだから、早めに見に行った方がいいかも。\nコタロー: え、そうなの？それは急がなきゃ！タロー、教えてくれてありがとう。\nタロー: どういたしまして。楽しんできてね。\nコタロー: うん、ありがとう！感想、また後で話すね。\nタロー: うん、楽しみにしてるよ。\n```"
        },
        {
            "input": "新入社員のオンボーディングをしています。以下の項目を、新入社員が早く馴染めるようにするために、どのような順番で行うのが良いか、提案してください。\n\n- 社内見学\n- 自己紹介\n- 契約書へのサイン\n- 社員証の配布\n- 福利厚生の説明",
            "output": "新入社員が早く会社に馴染めるよう、オンボーディングの項目を以下の順番で行うことを提案します。\n\n1. **自己紹介**: まずは、新入社員と既存社員がお互いを知る機会を設けましょう。新入社員には、自分の名前、出身地、趣味などを簡単に話してもらい、既存社員も自己紹介をすることで、親睦を深めます。これにより、新入社員の緊張をほぐし、リラックスした雰囲気を作ることができます。\n2. **社内見学**: 次に、新入社員に社内を案内します。各部署の役割や業務内容を説明しながら、実際に働く場所を見てもらうことで、会社全体のイメージを掴んでもらいます。また、社員食堂や休憩室などの設備も紹介し、安心して働ける環境であることを伝えます。\n3. **社員証の配布**: 社内見学の後、社員証を配布します。社員証は、会社の一員であることを示す大切なものです。社員証を身につけることで、新入社員の帰属意識を高め、モチベーション向上にも繋がります。\n4. **福利厚生の説明**: 続いて、福利厚生について説明します。社会保険や年金制度、休暇制度、各種手当など、社員が利用できる制度について詳しく説明することで、新入社員の不安を解消し、安心して働ける環境を整えます。\n5. **契約書へのサイン**: 最後に、雇用契約書の内容を確認し、サインをもらいます。契約書には、労働条件や給与、勤務時間などが記載されています。新入社員が内容を十分に理解した上でサインできるよう、丁寧に説明し、質問にもしっかり答えることが大切です。\n\nこの順番で行うことで、新入社員はスムーズに会社に馴染むことができ、早期の戦力化が期待できます。また、オンボーディングは、新入社員だけでなく、既存社員にとっても、新入社員を理解し、受け入れるための良い機会となります。お互いに協力し合い、より良い職場環境を作っていきましょう。"
        },
        {
            "input": "あなたは以下の道具を持っています。これらの道具を必要なだけ使って、今日の昼ごはんを準備してください。\n\n- フライパン\n- 包丁\n- まな板\n- ざる\n- ボウル\n- 菜箸\n- おたま\n- しゃもじ\n- トング",
            "output": "はい、承知しました。以下の手順で昼食を準備します。\n\n**メニュー：野菜たっぷり焼きそば**\n\n**使用する道具：** フライパン、包丁、まな板、ざる、ボウル、菜箸、トング\n\n**手順：**\n\n1. **野菜の下ごしらえ**: \n    *   キャベツ、にんじん、玉ねぎなどの野菜を、包丁とまな板を使って食べやすい大きさに切ります。\n    *   もやしは、ざるに入れて水洗いし、水気を切っておきます。\n2. **麺の下ごしらえ**: \n    *   焼きそばの麺を袋から取り出し、ボウルに入れます。\n    *   麺がくっついている場合は、ほぐしておきます。\n3. **炒める**: \n    *   フライパンを中火で熱し、油をひきます。\n    *   まず、切った野菜（キャベツ、にんじん、玉ねぎなど）をフライパンに入れ、菜箸で炒めます。\n    *   野菜に火が通ってきたら、もやしを加えます。\n    *   さらに、麺を加えて、トングを使って全体を混ぜ合わせながら炒めます。\n4. **味付け**: \n    *   麺と野菜がよく混ざったら、ソース、塩、こしょうなどで味付けします。\n    *   全体に味がなじむように、トングでよく混ぜ合わせます。\n5. **盛り付け**: \n    *   お皿に盛り付けて、完成です。\n\nお好みで、青のりや紅ショウガをトッピングしても美味しいです。また、豚肉やシーフードミックスなどを加えれば、より豪華な焼きそばになります。冷蔵庫にある食材と相談しながら、自由にアレンジしてみてください。楽しいランチタイムをお過ごしください！"
        }
    ],
    "事実確認": [
        {
            "input": "日本の初代内閣総理大臣は誰ですか？",
            "output": "日本の初代内閣総理大臣は、伊藤博文です。1885年12月22日に就任しました。"
        },
        {
            "input": "アメリカ合衆国の首都はどこですか？",
            "output": "アメリカ合衆国の首都はワシントンD.C.です。ワシントンD.C.は、どの州にも属さない特別区（District of Columbia）です。"
        },
        {
            "input": "現在、火星に人類はいますか？",
            "output": "いいえ、現在火星に人類は存在しません。2023年11月現在、火星には無人探査機が送り込まれ、調査が行われていますが、有人探査はまだ実現していません。しかし、アメリカのNASAやSpaceXなどの機関が、将来的な有人火星探査を目指して、研究開発を進めています。"
        }
    ],
    "意見・推測": [
        {
            "input": "将来、AIは人間の仕事を奪うと思いますか？",
            "output": "将来、AIが人間の仕事を奪う可能性はあると思います。特に、単純作業や定型業務はAIに代替される可能性が高いでしょう。例えば、工場のライン作業やデータ入力、受付業務などは、AIが得意とする分野です。しかし、AIは人間の仕事を奪うだけでなく、新しい仕事を生み出す可能性もあります。AIの開発や運用、AIを活用した新しいサービスなど、AIに関連する仕事は今後ますます増えていくと考えられます。また、AIにはできない、人間ならではの仕事もたくさんあります。例えば、創造性を発揮する仕事や、人と人とのコミュニケーションが重要な仕事、複雑な判断を必要とする仕事などは、AIに代替することは難しいでしょう。したがって、AIの発展は、人間の仕事のあり方を大きく変える可能性がありますが、すべての仕事がなくなるわけではないと考えます。人間は、AIと共存しながら、AIにはできない仕事や、AIを活用した新しい仕事にシフトしていくのではないでしょうか。"
        },
        {
            "input": "今後、地球温暖化はさらに深刻化すると思いますか？",
            "output": "私は、地球温暖化は今後さらに深刻化する可能性が高いと考えています。その理由は、現在の温室効果ガスの排出削減ペースでは、地球の気温上昇を抑えるには不十分だからです。IPCC（気候変動に関する政府間パネル）の報告書によると、産業革命前からの気温上昇を1.5℃未満に抑えるためには、2030年までに温室効果ガスの排出量を2010年比で約45%削減し、2050年頃には実質ゼロにする必要があります。しかし、現状の各国の取り組みでは、この目標を達成することは難しいと予測されています。また、地球温暖化の影響はすでに世界各地で現れており、異常気象や海面上昇などの問題が深刻化しています。これらの影響は、今後さらに拡大すると予想されます。地球温暖化を食い止めるためには、世界各国が協力して、温室効果ガスの排出削減に向けた取り組みを強化する必要があります。具体的には、再生可能エネルギーの導入拡大、省エネルギーの推進、森林保護などの対策が重要です。また、私たち一人ひとりも、日常生活の中でできることから取り組むことが大切です。例えば、公共交通機関の利用、節電、節水、ごみの削減など、小さな心がけが地球温暖化対策につながります。地球温暖化は、私たち人類にとって大きな脅威です。しかし、今ならまだ、その影響を最小限に抑えることができます。未来の世代のために、今こそ行動を起こす時だと考えます。"
        },
        {
            "input": "彼は昔、ある出来事をきっかけにひどく自暴自棄になり、当時は周りの人々も困惑していたようだった。しかし、数年前に彼女と出会ってからは、まるで別人のように穏やかになった。彼女は彼にとってどのような存在なのだろうか？",
            "output": "彼女は彼にとって、人生を大きく変えるきっかけを与えてくれた、かけがえのない存在なのだと思います。自暴自棄になっていた彼を救い、穏やかな人間性を取り戻させてくれたのですから、彼にとって彼女は、まさに「運命の人」と言えるのではないでしょうか。彼女の存在は、彼に「生きる希望」や「心の安らぎ」を与えてくれたのだと推測します。また、彼女は彼の良き理解者であり、精神的な支えでもあるのでしょう。彼女との出会いが、彼に「自分らしく生きることの大切さ」を気づかせてくれたのかもしれません。彼女は、彼にとって、人生の伴侶であり、心の拠り所であり、希望の光でもある、そんな存在なのだと思います。"
        }
    ],
    "評価・レビュー": [
        {
            "input": "映画「タイタニック」のレビューを書いてください。",
            "output": "映画「タイタニック」は、1912年に実際に起きた豪華客船タイタニック号の沈没事故を基に、身分違いの恋に落ちたジャックとローズの悲恋を描いたラブストーリーです。1997年に公開され、世界中で大ヒットを記録しました。\n\n**評価：★★★★★（5/5）**\n\n**良い点:**\n\n*   **迫力満点の映像**: 沈みゆくタイタニック号の様子は、CG技術を駆使してリアルに再現されており、観客を圧倒します。特に、船体が真っ二つに折れ、沈没していくシーンは、映画史に残る名シーンと言えるでしょう。\n*   **感動的なストーリー**: ジャックとローズの身分違いの恋は、観客の心を強く揺さぶります。二人の愛の行方を見守りながら、タイタニック号の悲劇を追体験することができます。\n*   **心に残る音楽**: セリーヌ・ディオンが歌う主題歌「My Heart Will Go On」をはじめ、映画全体を彩る音楽が、ストーリーをより一層感動的にしています。\n*   **豪華キャスト**: レオナルド・ディカプリオ、ケイト・ウィンスレットをはじめ、実力派俳優陣の演技も見どころです。\n\n**悪い点:**\n\n*   **上映時間が長い**: 約3時間という上映時間は、人によっては長く感じるかもしれません。\n*   **悲劇的な結末**: 史実に基づいているため、ハッピーエンドではありません。悲しい結末が苦手な人には、つらいかもしれません。\n\n**改善点:**\n\n*   上映時間をもう少し短くしても良かったかもしれません。\n\n**総合評価:**\n\n「タイタニック」は、映像、ストーリー、音楽、キャスト、すべてにおいて高いクオリティを誇る、映画史に残る傑作です。何度見ても感動できる、不朽の名作と言えるでしょう。まだ見たことがない人は、ぜひ一度ご覧になることをおすすめします。"
        },
        {
            "input": "このレストランの料理はどうでしたか？",
            "output": "このレストランの料理は、全体的にとても美味しかったです。特に、前菜でいただいたカルパッチョは、新鮮な魚介類と特製のソースが絶妙にマッチしていて、感動的な味わいでした。メインのパスタは、アルデンテに茹で上げられた麺に、濃厚なソースが絡み、本格的なイタリアンを楽しむことができました。デザートのティラミスも、甘すぎず、ほろ苦いエスプレッソの風味が効いていて、大人の味わいでした。また、ワインの種類も豊富で、料理とのマリアージュを楽しむことができました。\n\n**評価：★★★★☆（4.5/5）**\n\n**良い点:**\n\n*   どの料理も、素材の良さが活かされており、丁寧に調理されていると感じました。\n*   盛り付けも美しく、目でも楽しむことができました。\n*   スタッフのサービスも丁寧で、気持ちよく食事をすることができました。\n\n**悪い点:**\n\n*   人気店のため、予約が取りづらいのが難点です。\n*   価格帯がやや高めなので、特別な日に利用したいお店です。\n\n**改善点:**\n\n*   ランチメニューなど、もう少しリーズナブルな価格帯のメニューがあると嬉しいです。\n\n**総合評価:**\n\n味、サービス、雰囲気ともに満足できるレストランでした。価格帯はやや高めですが、特別な日にまた利用したいと思います。"
        },
        {
            "input": "この小説の感想を教えてください。",
            "output": "この小説は、現代社会の闇を鋭く描き出した、非常に読み応えのある作品でした。主人公の心理描写が巧みで、彼の苦悩や葛藤が手に取るように伝わってきました。また、ストーリー展開もスリリングで、最後まで飽きさせません。特に、ラストのどんでん返しには驚愕しました。現代社会に生きる私たちにとって、多くの示唆を与えてくれる作品だと思います。\n\n**評価：★★★★☆（4/5）**\n\n**良い点:**\n\n*   **現代社会への問題提起**: この小説は、現代社会が抱える様々な問題を、鋭い視点で描き出しています。読者に、現代社会のあり方について、改めて考えさせるきっかけを与えてくれます。\n*   **巧みな心理描写**: 主人公をはじめとする登場人物たちの心理描写が非常に巧みです。彼らの心の動きが手に取るように伝わってきて、物語に深く没入することができます。\n*   **スリリングな展開**: ストーリー展開がスリリングで、先が読めない面白さがあります。読者を飽きさせず、最後まで一気に読ませる力があります。\n\n**悪い点:**\n\n*   **救いのない結末**: 人によっては、後味の悪い結末だと感じるかもしれません。もう少し救いのある結末でも良かったのではないかと思います。\n*   **やや難解な表現**: 一部、難解な表現や専門用語が使われており、読みにくいと感じる部分がありました。\n\n**改善点:**\n\n*   難解な表現や専門用語には、注釈をつけるなどの工夫があると、より多くの読者が楽しめると思います。\n\n**総合評価:**\n\n多少の難点はありますが、現代社会に生きる私たちにとって、一読の価値がある作品だと思います。特に、社会問題に関心のある方や、深い人間ドラマが好きな方におすすめです。"
        }
    ],
    "変換・言い換え": [
        {
            "input": "「犬も歩けば棒に当たる」ということわざを、別の表現で言い換えてください。",
            "output": "「犬も歩けば棒に当たる」ということわざは、「何か行動を起こせば、思いがけない幸運に出会うことがある」という意味です。似た意味のことわざとしては、「下手な鉄砲も数撃ちゃ当たる」、「果報は寝て待て」、「継続は力なり」などがあります。また、「失敗を恐れずに行動することが大切だ」という意味で、「案ずるより産むが易し」ということわざも近い意味と言えるでしょう。"
        },
        {
            "input": "「彼はとても頭が良い」という文を、別の言い方で表現してください。",
            "output": "「彼はとても頭が良い」という文は、以下のように言い換えることができます。\n\n*   彼は非常に**聡明**だ。\n*   彼は**頭脳明晰**だ。\n*   彼は**抜群に頭の回転が速い**。\n*   彼は**並外れた知性の持ち主**だ。\n*   彼は**才気煥発**だ。\n\nこれらの表現は、いずれも「頭が良い」という意味をより強調したり、具体的に表現したりしています。"
        },
        {
            "input": "「愛飲」を簡単な表現に言い換えて。",
            "output": "「愛飲」は、「いつも好んで飲むこと」や「普段からよく飲んでいること」と言い換えることができます。例えば、「彼はコーヒーを愛飲している」は、「彼はいつもコーヒーを好んで飲んでいる」や「彼は普段からよくコーヒーを飲んでいる」と言い換えることができます。"
        }
    ],
    "要約": [
        {
            "input": "次の文章を要約してください。\n\n「桃太郎は、おじいさんとおばあさんから黍団子をもらって、鬼ヶ島へ鬼退治に出かけました。道中、イヌ、サル、キジと出会い、黍団子を渡して家来にしました。鬼ヶ島では、鬼たちと戦い、見事勝利しました。そして、鬼たちが奪った宝物を取り返し、村へ帰りました。」",
            "output": "桃太郎は、おじいさんとおばあさんからもらった黍団子でイヌ、サル、キジを家来にし、鬼ヶ島へ向かいました。鬼ヶ島では鬼たちを退治し、宝物を取り返して村へ帰りました。 **(75文字)**"
        },
        {
            "input": "次のニュース記事を要約してください。\n\n「本日、東京で桜の開花が発表されました。平年より5日早く、昨年より3日遅い開花となりました。気象庁によると、今週は暖かい日が続くため、来週末には満開となる見込みです。」",
            "output": "本日、東京で桜の開花が発表されました。平年より5日早く、昨年より3日遅い開花です。今週は暖かい日が続くため、来週末に満開となる見込みです。 **(66文字)**"
        },
        {
            "input": "これは演劇のシナリオです。\n要約: ある日、主人公の父親が突然亡くなり、主人公は父親の遺産を相続することになります。しかし、父親の遺産は、父が生前に作った演劇の脚本でした。\n主人公は、父親が作った演劇の脚本を読み始めると、父親の過去の出来事が明らかになります。そして父の過去が、主人公の人生に大きな影響を与えていきます。\n\n以上の文章をCM風に言い換えてください。",
            "output": "父が遺した、一冊の戯曲。それは、父の過去への扉だった。知られざる父の想いが、息子の人生を揺さぶる。時を超えた父と息子の物語が、今、幕を開ける！ **(68文字)**"
        }
    ],
    "翻訳": [
        {
            "input": "次の文章を英語に翻訳してください。\n\n今日は良い天気ですね。",
            "output": "The weather is nice today, isn't it? **(直訳: 今日は天気が良いですね、そう思いませんか？)**\n\n**その他の表現:**\n\n*   It's a beautiful day today. (今日は美しい日ですね。)\n*   It's a lovely day, isn't it? (今日は素晴らしい日ですね、そう思いませんか？)\n*   We're having a nice day today. (今日は良い日ですね。)"
        },
        {
            "input": "次の文章を日本語に翻訳してください。\n\nI have a dream that my four little children will one day live in a nation where they will not be judged by the color of their skin but by the content of their character.",
            "output": "私には夢があります。それは、私の4人の幼い子どもたちが、いつの日か、肌の色によってではなく、人格そのものによって評価される国に住むという夢です。 **(マーティン・ルーサー・キング・ジュニア牧師の有名な演説の一節)**"
        },
        {
            "input": "次の文章を中国語に翻訳してください。\n\n明日は雨が降るそうです。",
            "output": "听说、明天要下雨。(Tīng shuō, míngtiān yào xiàyǔ.) **(直訳: 聞くところによると、明日は雨が降るそうです。)**\n\n**その他の表現:**\n\n*   明天好像要下雨。(Míngtiān hǎoxiàng yào xiàyǔ.) (明日は雨が降るようです。)\n*   天气预报说明天下雨。(Tiānqì yùbào shuō míngtiān xiàyǔ.) (天気予報によると明日は雨です。)"
        }
    ],
    "感情分析": [
        {
            "input": "次の文章の感情を、ポジティブ、ネガティブ、ニュートラルのいずれかで答えてください。\n\n今日はとても楽しかったです。",
            "output": "ポジティブ\n\n「とても楽しかった」という表現から、書き手が今日の出来事に対して、非常に肯定的な感情を抱いていることがわかります。「楽しい」という言葉は、喜びや満足感を表すポジティブな感情表現です。"
        },
        {
            "input": "次の文章の感情を、ポジティブ、ネガティブ、ニュートラルのいずれかで答えてください。\n\n今日は最悪な一日でした。",
            "output": "ネガティブ\n\n「最悪な一日」という表現から、書き手が今日の出来事に対して、非常に否定的な感情を抱いていることがわかります。「最悪」という言葉は、不満や失望、怒りなどのネガティブな感情を強調する表現です。"
        },
        {
            "input": "次の文章の感情を、ポジティブ、ネガティブ、ニュートラルのいずれかで答えてください。\n\n今日は普通の一日でした。",
            "output": "ニュートラル\n\n「普通の一日」という表現からは、書き手が今日の出来事に対して、特に強い感情を抱いていないことがわかります。「普通」という言葉は、良くも悪くもない、平常な状態を表す言葉であり、感情的には中立（ニュートラル）な状態を示しています。"
        }
    ],
    "対話": [
        {
            "input": "ユーザー: こんにちは！\nアシスタント: こんにちは！今日はどうされましたか？",
            "output": "ユーザー: 今日の天気を教えてください。\nアシスタント: かしこまりました。どちらの地域の天気をお知りになりたいですか？"
        },
        {
            "input": "ユーザー: あなたの名前は何ですか？\nアシスタント: 私の名前はアシスタントです。",
            "output": "ユーザー: アシスタントさん、好きな食べ物は何ですか？\nアシスタント: 私はAIですので、食べ物を食べることはできません。しかし、もし食べることができるなら、世界中の美味しいものを試してみたいと思います。特に、日本の寿司やイタリアのピザに興味があります。"
        },
        {
            "input": "ユーザー: 今日のニュースを教えてください。\nアシスタント: 今日の主なニュースは、〇〇です。",
            "output": "ユーザー: 〇〇について詳しく教えてください。\nアシスタント: かしこまりました。〇〇についてですね。〇〇は、[日付]に[場所]で発生した[事件・出来事]です。この[事件・出来事]により、[影響]が出ています。[関係者]は、[対応]を行っています。現在、[状況]となっています。"
        }
    ],
    "ソート・並び替え": [
        {
            "input": "次の数字を小さい順に並び替えてください。\n\n5, 2, 8, 1, 9",
            "output": "1, 2, 5, 8, 9\n\n**並び替えのルール:** 数字を小さい順に並べ替えます。数字の大きさを比較して、左から順に並べていきます。"
        },
        {
            "input": "次の果物を五十音順に並び替えてください。\n\nりんご、バナナ、みかん、いちご、ぶどう",
            "output": "いちご、ぶどう、バナナ、みかん、りんご\n\n**並び替えのルール:** 果物の名前を五十音順に並べ替えます。最初の文字が同じ場合は、次の文字で比較します。これを、単語の終わりまで繰り返します。"
        },
        {
            "input": "次の国名を面積が大きい順に並び替えてください。\n\nロシア、カナダ、中国、アメリカ、ブラジル",
            "output": "ロシア、カナダ、中国、アメリカ、ブラジル\n\n**並び替えのルール:** 国の面積が大きい順に並べ替えます。国名は、面積の大きい順に、左から並べていきます。それぞれの国の面積は、以下の通りです。\n\n*   ロシア: 約17,098,242平方キロメートル\n*   カナダ: 約9,984,670平方キロメートル\n*   中国: 約9,596,961平方キロメートル\n*   アメリカ: 約9,372,610平方キロメートル\n*   ブラジル: 約8,515,767平方キロメートル"
        }
    ],
    "比較": [
        {
            "input": "「犬」と「猫」の違いを3つ挙げてください。",
            "output": "「犬」と「猫」の違いを3つ挙げると、以下のようになります。\n\n| 項目 | 犬 | 猫 |\n|---|---|---| \n| **分類** | 哺乳綱食肉目イヌ科イヌ属 | 哺乳綱食肉目ネコ科ネコ属 |\n| **性格** | 社交的で、飼い主に従順な傾向がある | 独立心が強く、気ままな性格の傾向がある |\n| **食性** | 雑食性（肉、魚、野菜など何でも食べる） | 肉食性（肉や魚を好む） |\n\nこれらの違いは、犬と猫の進化の過程や、人間との関わりの歴史の違いに由来しています。犬は古くから人間のパートナーとして、狩猟や牧畜などを手伝ってきました。そのため、人間とのコミュニケーション能力が高く、飼い主に従順な性格に進化したと考えられます。一方、猫は単独で狩りをする動物であり、独立心が強い性格をしています。近年では、猫もペットとして人気が高まっており、人間との関わり方も変化してきています。"
        },
        {
            "input": "「日本」と「アメリカ」の違いを3つ挙げてください。",
            "output": "「日本」と「アメリカ」の違いはたくさんありますが、ここでは代表的な違いを3つ挙げて、表形式で説明します。\n\n| 項目 | 日本 | アメリカ |\n|---|---|---| \n| **国土** | 島国であり、国土の約70%が山地 | 広大な国土を持ち、多様な地形が存在 |\n| **公用語** | 日本語 | 事実上の公用語は英語（法律で定められているわけではない） |\n| **政治体制** | 議会制民主主義（象徴天皇制） | 連邦共和制（大統領制） |\n\nこれらの違いは、両国の歴史や文化、社会構造などに深く関わっています。例えば、日本は島国であるため、古くから独自の文化が育まれてきました。一方、アメリカは移民によって建国された国であり、多様な文化が共存しています。また、日本は単一民族国家に近い社会構造を持っていますが、アメリカは多民族国家であり、人種のるつぼとも呼ばれています。"
        },
        {
            "input": "「夏」と「冬」の違いを3つ挙げてください。",
            "output": "「夏」と「冬」の違いはたくさんありますが、ここでは代表的な違いを3つ挙げて、表形式で説明します。\n\n| 項目 | 夏 | 冬 |\n|---|---|---| \n| **気温** | 高い | 低い |\n| **日照時間** | 長い | 短い |\n| **服装** | 半袖、短パンなど、薄着 | コート、マフラーなど、厚着 |\n\nこれらの違いは、地球の地軸の傾きと、太陽との位置関係によって生じます。夏は、太陽の光が地球に当たる角度が大きいため、気温が高く、日照時間が長くなります。一方、冬は、太陽の光が地球に当たる角度が小さいため、気温が低く、日照時間が短くなります。また、夏と冬では、服装だけでなく、食べ物やイベントなど、人々の生活の様々な面で違いが見られます。"
        }
    ],
    "分類・カテゴライズ": [
        {
            "input": "次の文章を読んで、筆者が伝えたいメッセージがより近い項目を以下から選んでください。\n\n「彼はバブル期に大金持ちになり豊かな生活に日夜明け暮れたが、バブル崩壊後にほとんどの資産と職を失った。彼女に出会ったのは、そんな人生が真っ暗になってしまっていたときだった。彼は現在アルバイトで貧しいながらも生計を立てているが、それでも彼は今が一番豊かだと言っている。」\n\n- お金が全てではない\n- 一度得たものは簡単に手放してはいけない",
            "output": "筆者が伝えたいメッセージは「**お金が全てではない**」の方がより近いです。\n\n**理由:**\n\n*   文章の後半で、彼はバブル崩壊後、貧しい生活を送りながらも「今が一番豊かだ」と言っています。これは、彼がお金よりも大切なもの（彼女との出会い）を得たことを示唆しています。\n*   「一度得たものは簡単に手放してはいけない」というメッセージは、文章中では読み取れません。\n\nこの文章は、お金や物質的な豊かさだけが幸せではないということを伝えようとしていると考えられます。真の豊かさとは、愛や心の充足感など、お金では買えないものの中にあるというメッセージが込められているのではないでしょうか。"
        },
        {
            "input": "以下の単語を、性質がより近いグループに分類してください。\n\n- トマト\n- りんご\n- バナナ\n- ほうれん草\n- キャベツ\n- ぶどう",
            "output": "以下のように分類できます。\n\n| グループ | 単語 |\n|---|---| \n| **果物** | りんご、バナナ、ぶどう |\n| **野菜** | トマト、ほうれん草、キャベツ |\n\n**分類の基準:**\n\n*   **果物**: 木や草になる、食用となる果実。甘味があるものが多い。\n*   **野菜**: 食用となる草本植物の総称。主食に対して、副食（おかず）となるもの。\n\nトマトは、植物学上は果物に分類されますが、食文化上は野菜として扱われることが多いため、ここでは野菜に分類しています。"
        },
        {
            "input": "次の文章は、何に関する説明ですか？選択肢から最も適したものを選んでください。\n\n「その物質は無色透明の液体で、特有の刺激臭があります。水によく溶け、酸性を示します。工業的に重要な物質であり、様々な用途に用いられています。」\n\n- 水\n- エタノール\n- 塩酸\n- アンモニア",
            "output": "この文章は「**塩酸**」に関する説明です。\n\n**理由:**\n\n*   「無色透明の液体」「特有の刺激臭」「水によく溶け、酸性を示す」という記述は、いずれも塩酸の特徴と一致します。\n*   「工業的に重要な物質であり、様々な用途に用いられています」という点も、塩酸の特徴と一致します。\n\n他の選択肢は、以下の理由で不適切です。\n\n*   **水**: 無臭であり、中性です。\n*   **エタノール**: 特有の臭気がありますが、酸性を示しません。\n*   **アンモニア**: 特有の刺激臭があり、水によく溶けますが、アルカリ性を示します。"
        },
    ],
    "ロールプレイ": [
        {
            "input": "これは悟りを開いたブッダとの会話です。知恵と愛に満ちていた会話をしてください。\n私: どうしたら、より大きな平和と平穏を得ることができますか？\nブッダ:",
            "output": "ブッダ: 「より大きな平和と平穏を求める、その心の中にこそ、すでに平和と平穏があるのです。あなたは、平和と平穏を外に求めていますが、それはあなたの内側にあります。自分自身の心と向き合い、心を静めることで、平和と平穏を得ることができるのです。呼吸に意識を集中し、今この瞬間に心を留めてみてください。過去への執着や未来への不安を手放し、今を生きるのです。そうすれば、あなたの心は自然と穏やかになり、平和と平穏に満たされるでしょう。」"
        },
        {
            "input": "あなたはカスタマーサポートの担当者です。顧客からの問い合わせに、丁寧かつ親切に対応してください。\n\n顧客: 昨日購入した商品が不良品のようです。交換してもらえますか？",
            "output": "ご連絡ありがとうございます。ご購入いただいた商品に不良があったとのこと、大変申し訳ございません。ご迷惑をおかけして、誠に申し訳ございません。商品の交換を承りますので、以下の点について、詳しくお聞かせいただけますでしょうか？\n\n*   ご購入いただいた商品名\n*   ご購入日時\n*   ご購入店舗名\n*   商品の具体的な状態（どのように不良なのか、できるだけ詳しくお聞かせください）\n*   お客様のご連絡先（お電話番号、メールアドレスなど）\n\nお手数をおかけしますが、よろしくお願いいたします。商品の状態を確認させていただき、迅速に対応させていただきます。また、何かご不明な点がございましたら、お気軽にお問い合わせください。"
        },
        {
            "input": "あなたは、友人に映画を勧める立場です。最近見た映画のあらすじと感想を、友人に伝えてください。",
            "output": "ねえねえ、最近「君の名は。」っていう映画を見たんだけど、めっちゃ良かったから聞いて！東京に住む男子高校生の瀧（たき）と、田舎町に住む女子高生の三葉（みつは）が、夢の中で入れ替わっちゃうっていう話なんだけど、もうストーリーが本当に面白くて、最初から最後までドキドキしっぱなしだったよ！映像もすごく綺麗で、特に彗星が落ちるシーンは圧巻だった！音楽も良くて、RADWIMPSの曲が映画の世界観にぴったりだったよ。最初はコメディっぽい感じなんだけど、だんだんシリアスな展開になっていって、最後はもう感動して泣いちゃった。瀧と三葉が、お互いのことを必死に探す姿には、本当に胸が締め付けられたよ。SFっぽい要素もあるんだけど、恋愛映画としてもすごく良くて、見終わった後も、しばらく余韻に浸っちゃった。これは絶対見た方がいいよ！私ももう一回見たいくらい！"
        },
    ],
    "生成・作成": [
        {
            "input": "「全力、安全、活用、軽減」の言葉を順に全て使って、1つの文を書いてください",
            "output": "私たちは、事故のリスクを**軽減**するために、**安全**を最優先に考え、持てる技術を**活用**し、業務に**全力**で取り組みます。"
        },
        {
            "input": "「生き物デザイナー」という職業があります。これは、自分が考えたオリジナルの生き物をデザインし、実際にDNAを編集して作り出す仕事です。あなたが生き物デザイナーである場合、どんな生き物を作りたいですか？また、その生き物が持つ特徴や能力について説明してください。",
            "output": "私が生き物デザイナーであれば、砂漠地帯の過酷な環境でも生息できる「ラクダドリ」という生き物を作りたいです。この生き物は、ラクダと鳥類の特徴を併せ持っています。\n\n**特徴・能力:**\n\n*   **コブ:** ラクダのように背中にコブがあり、長期間の乾燥に耐えられるよう、大量の水分を蓄えることができます。また、体温調節機能も備えており、砂漠の激しい気温差にも対応できます。\n*   **羽毛:** 鳥類のように全身が羽毛で覆われていますが、この羽毛は特殊な構造をしており、太陽光を反射して体温の上昇を防ぐと同時に、夜間の冷気から身を守る断熱材の役割も果たします。また、羽毛の色は砂漠に溶け込むような保護色になっています。\n*   **脚:** 脚は長く、ダチョウのように速く走ることができます。これにより、広大な砂漠を効率的に移動し、捕食者から逃れることができます。\n*   **嘴（くちばし）:** 嘴は鋭く、サボテンなどの多肉植物を突き破り、中の水分を摂取することができます。また、昆虫や小動物を捕食する際にも役立ちます。\n*   **視覚:** 優れた視覚を持ち、遠くのオアシスや獲物を発見することができます。\n\nラクダドリは、その優れた適応能力により、砂漠という過酷な環境でも生き抜くことができます。また、人間が砂漠で生活する上でも、移動手段や食料源として役立つ可能性があります。さらに、ラクダドリの持つ特殊な能力を研究することで、砂漠化の進む地域での食料問題や水不足問題の解決に繋がるかもしれません。"
        },
        {
            "input": "次の物語の展開を予想してみましょう。\n\nある日、主人公のもとに不思議な手紙が届きました。手紙には「今夜の満月に、森の奥深くにある洞窟に来てください。あなたを待っています。」と書かれていました。主人公はその手紙に従い、夜中に洞窟の入り口にたどり着きました。中に入ると、謎めいた人物と出会い……。",
            "output": "主人公が洞窟の中に入ると、薄暗い空間の奥に、ローブをまとった謎めいた人物が立っていました。その人物は静かに、しかし力強い声で言いました。「よく来たな、選ばれし者よ。私はこの洞窟を守る者。お前が来るのを長い間待っていた。」\n\n主人公は恐る恐る尋ねました。「私を呼んだのはあなたですか？なぜ私をここに？」謎めいた人物は答えました。「お前を呼んだのは、この世界を救うためだ。この洞窟の奥には、古の秘宝が眠っている。その秘宝は、世界を闇から救う力を持っている。しかし、秘宝を手に入れるためには、三つの試練を乗り越えなければならない。」\n\n「試練、ですか？」主人公は戸惑いながらも、決意を込めて尋ねました。「その試練とは、一体どのようなものなのですか？」謎めいた人物は、洞窟の奥へと続く道を指差しました。「最初の試練は、勇気を示すことだ。この先には、お前の恐怖心を試す罠が仕掛けられている。己の恐怖に打ち勝ち、進むべき道を見極めよ。」\n\n主人公は意を決して、洞窟の奥へと進んでいきました。洞窟の中は暗く、不気味な雰囲気が漂っていました。どこからか、獣のうなり声のようなものも聞こえてきます。しばらく進むと、主人公の前に大きな穴が現れました。穴の底は見えず、向こう岸に渡るには、細い一本橋を渡るしかありません。しかも、橋の下には、巨大なクモが巣を張って待ち構えていました。「これが最初の試練か…」主人公は恐怖に震えながらも、勇気を振り絞って、橋を渡り始めました。\n\n【この後の展開の例】\n\n*   主人公は、クモの巣にかかってしまうが、なんとか脱出し、向こう岸にたどり着く。その過程で、恐怖心を克服する。\n*   主人公は、クモと戦い、見事勝利する。その過程で、自分の内に秘めた力に気づく。\n*   主人公は、クモを説得し、橋を渡らせてもらう。その過程で、言葉の力、対話の重要性に気づく。\n\n果たして、主人公は最初の試練を乗り越えることができるのでしょうか？そして、残りの二つの試練とは？主人公は無事に秘宝を手に入れ、世界を救うことができるのでしょうか？"
        }
    ],
    "ガイド・アドバイス": [
        {
            "input": "新入社員のオンボーディングをしています。以下の項目を、新入社員が早く馴染めるようにするために、どのような順番で行うのが良いか、提案してください。\n\n- 社内見学\n- 自己紹介\n- 契約書へのサイン\n- 社員証の配布\n- 福利厚生の説明",
            "output": "新入社員が早く会社に馴染めるように、オンボーディングは以下の順番で行うことを提案します。\n\n1. **自己紹介**: \n    *   **目的**: 新入社員と既存社員の間の最初の接点を作り、新入社員がリラックスして、チームに溶け込めるようにするためです。\n    *   **方法**: まず、新入社員から自己紹介をしてもらい、その後、チームメンバーも自己紹介を行います。趣味や経歴などを簡単に共有することで、お互いの理解を深め、会話のきっかけを作ることができます。また、新入社員が質問しやすい雰囲気を作ることも大切です。\n2. **社内見学**: \n    *   **目的**: 新入社員が社内の環境や設備を理解し、スムーズに業務を開始できるようにするためです。\n    *   **方法**: 社内の各部署や設備（休憩室、トイレ、会議室など）を案内します。その際、各部署の役割や業務内容を簡単に説明すると良いでしょう。また、新入社員が迷子にならないように、地図を渡したり、よく使う場所への行き方を丁寧に教えたりすることも効果的です。\n3. **社員証の配布**: \n    *   **目的**: 新入社員に帰属意識を持ってもらい、会社の一員としての自覚を促すためです。\n    *   **方法**: 社員証を配布し、その使い方（入退館の方法など）を説明します。社員証は、会社の一員であることを示す重要なアイテムです。配布することで、新入社員のモチベーションアップにも繋がります。\n4. **福利厚生の説明**: \n    *   **目的**: 新入社員が利用できる福利厚生制度を理解し、安心して働ける環境を整えるためです。\n    *   **方法**: 会社が提供する福利厚生制度（社会保険、休暇制度、健康診断、各種手当など）について詳しく説明します。新入社員が制度を有効活用できるように、申請方法や利用条件なども丁寧に説明しましょう。また、質問があれば、その場で回答することで、新入社員の不安を解消することができます。\n5. **契約書へのサイン**: \n    *   **目的**: 雇用契約の内容を確認し、労使間のトラブルを未然に防ぐためです。\n    *   **方法**: 最後に、雇用契約書の内容を丁寧に説明し、新入社員にサインをもらいます。契約書は、労働条件や給与、勤務時間など、重要な事項が記載された書類です。新入社員が内容を十分に理解した上で、サインできるように、時間をかけて説明することが大切です。また、契約書の控えを新入社員に渡すことも忘れないようにしましょう。\n\nこの順番で行うことで、新入社員はスムーズに会社に馴染み、早期に戦力として活躍できるようになると考えられます。また、オンボーディングは、新入社員だけでなく、既存社員にとっても、チームワークを高める良い機会となります。新入社員を温かく迎え入れ、会社全体でサポートしていくことが重要です。"
        },
        {
            "input": "あなたは以下の道具を持っています。これらの道具を必要なだけ使って、今日の昼ごはんを準備してください。\n\n- フライパン\n- 包丁\n- まな板\n- ざる\n- ボウル\n- 菜箸\n- おたま\n- しゃもじ\n- トング",
            "output": "はい、喜んで！これらの道具を使って、美味しい昼ごはんを準備しましょう！今回は、手軽に作れる「野菜たっぷり焼きそば」と「簡単中華スープ」の献立を提案します。調理時間は約20分です。\n\n**メニュー:**\n\n*   野菜たっぷり焼きそば\n*   簡単中華スープ\n\n**材料 (2人分):**\n\n*   焼きそば麺: 2玉\n*   豚肉 (薄切り): 100g\n*   キャベツ: 1/8個\n*   にんじん: 1/3本\n*   もやし: 1/2袋\n*   ニラ: 1/4束\n*   サラダ油: 大さじ1\n*   塩・こしょう: 少々\n*   焼きそばソース: 大さじ4\n*   鶏がらスープの素: 小さじ2\n*   水: 400ml\n*   ごま油: 小さじ1\n*   お好みで: 卵、青のり、紅しょうが\n\n**調理手順:**\n\n1. **下準備**: \n    *   **包丁**と**まな板**を使って、キャベツはざく切り、にんじんは短冊切り、ニラは4〜5cm幅に切ります。豚肉は食べやすい大きさに切ります。\n    *   **ボウル**に鶏がらスープの素と水を入れ、混ぜ合わせてスープを作ります。\n2. **焼きそばを作る**: \n    *   **フライパン**にサラダ油を熱し、豚肉を炒めます。色が変わったら、キャベツ、にんじん、もやしを加えて炒め合わせます。**菜箸**や**トング**を使うと便利です。\n    *   野菜がしんなりしたら、焼きそば麺を加えてほぐしながら炒めます。塩・こしょうで軽く味付けします。\n    *   麺がほぐれたら、焼きそばソースを加えて全体に絡めます。\n    *   最後にニラを加えてさっと炒め合わせ、火を止めます。\n    *   お皿に盛り付け、お好みで目玉焼きを乗せたり、青のりや紅しょうがを添えたりしてください。**しゃもじ**を使うと、きれいに盛り付けられます。\n3. **スープを作る**: \n    *   **おたま**を使って、1で作ったスープを**フライパン**に注ぎ入れ、中火にかけます。\n    *   沸騰したら弱火にし、ごま油を加えます。\n    *   器に注ぎ、お好みでネギを散らしてください。\n\nこれで、野菜たっぷり焼きそばと簡単中華スープの完成です！**ざる**は、今回使用しませんでしたが、野菜の水切りなどに便利です。洗い物も少なく、簡単に作れるので、ぜひ試してみてくださいね！"
        },
        {
            "input": "「スキーストック 選び方」と検索しているユーザーが他に興味を持ちそうな検索クエリをいくつかあげてください。",
            "output": "「スキーストック 選び方」と検索しているユーザーは、スキーを始めたばかりの人や、新しいストックの購入を検討している人だと考えられます。そのようなユーザーが他に興味を持ちそうな検索クエリを、以下にいくつか挙げてみます。\n\n**スキー関連:**\n\n*   **長さ・サイズ関連:**\n    *   スキーストック 長さ 選び方\n    *   スキーストック 長さ 測り方\n    *   スキーストック サイズ 選び方\n    *   スキーストック サイズ 調整\n*   **種類・タイプ関連:**\n    *   スキーストック 種類\n    *   スキーストック カーボン\n    *   スキーストック アルミ\n    *   スキーストック 伸縮式\n    *   スキーストック レーシング\n*   **ブランド・メーカー関連:**\n    *   スキーストック おすすめ メーカー\n    *   シナノ スキーストック 評判\n    *   レキ スキーストック 評判\n    *   スウィックス スキーストック 評判\n*   **価格・購入関連:**\n    *   スキーストック 価格\n    *   スキーストック 安い\n    *   スキーストック 中古\n    *   スキーストック 激安\n*   **その他:**\n    *   スキーストック メンテナンス\n    *   スキーストック リング 交換\n\n**スキー以外のウィンタースポーツ関連:**\n\n*   スノーボード ストック 選び方\n*   スノーシュー ストック 選び方\n\n**登山関連:**\n\n*   トレッキングポール 選び方\n*   登山 ストック 選び方\n\nこれらのクエリは、「スキーストック 選び方」と関連性が高く、ユーザーの関心を引きやすいと考えられます。また、これらのクエリを参考に、コンテンツを作成することで、ユーザーのニーズを満たし、満足度を高めることができるでしょう。"
        },
    ],
    "その他": [
        {
            "input": "「愛飲」を簡単な表現に言い換えて。",
            "output": "「愛飲」は、「いつも好んで飲んでいること」や「普段からよく飲んでいること」と言い換えることができます。例えば、「彼はコーヒーを愛飲している」は、「彼はいつもコーヒーを好んで飲んでいる」や「彼は普段からよくコーヒーを飲んでいる」と言い換えることができます。また、単に「よく飲む」と言い換えても良いでしょう。"
        },
        {
            "input": "「会場に女性がいた。」という文は「どこに女性がいた？」という疑問文に直すことができます。\nこのとき、 \"会場\" という単語は \"どこ\" という疑問詞タグを持ちます。疑問詞タグには いつ, どこ, だれ, なに, どれくらい があります。\n\nそれでは、「ドライバー」という単語はどのような疑問詞タグを持つでしょうか？ 全て選んでください。対応するものがない場合は\"なし\"と答えてください。",
            "output": "「ドライバー」という単語は、文脈によって「**だれ**」と「**なに**」の疑問詞タグを持ちます。\n\n**例:**\n\n*   **「だれ」の疑問詞タグを持つ場合:**\n    *   例文: 「タクシーの**ドライバー**が道を間違えた。」\n    *   この場合、「ドライバー」は「運転手」という人を指すため、「だれ」の疑問詞タグが対応します。\n*   **「なに」の疑問詞タグを持つ場合:**\n    *   例文: 「ネジを締めるのに**ドライバー**が必要です。」\n    *   この場合、「ドライバー」は「ねじ回し」という工具を指すため、「なに」の疑問詞タグが対応します。\n\nこのように、「ドライバー」という単語は、文脈によって意味が異なり、持つ疑問詞タグも変わります。"
        },
        {
            "input": "IMEとしてローマ字の入力を単語ごとにひらがなに変換し、漢字変換候補を提示します。\n\n```\ninputs:\nj, u, u, b, u, n, o, s, h, o, s, e, k, i\n\noutputs:\n- じゅう: 10, 十, 銃\n- ぶ: 部, 武, 分\n- の: の, ノ\n- しょせき: 書籍, 書跡, 諸関\n```",
            "output": "```\ninputs:\ns, h, i, n, k, a, n, s, e, n, h, a, y, a, i\n\noutputs:\n- しんかんせん: 新幹線, 神感線, 心間線\n- はやい: 速い, 早い, 疾い\n```\n\nローマ字入力「s, h, i, n, k, a, n, s, e, n, h, a, y, a, i」を単語ごとにひらがなに変換し、漢字変換候補を提示します。\n\n*   「s, h, i, n, k, a, n, s, e, n」は「しんかんせん」に変換され、漢字変換候補は「新幹線, 神感線, 心間線」です。\n*   「h, a, y, a, i」は「はやい」に変換され、漢字変換候補は「速い, 早い, 疾い」です。"
        },
        {
            "input": "次の文字列の括弧の対応関係をチェックし、不足している括弧を補完して、正しい文字列にしてください。\n\n[ < { } > [ { [ ] ",
            "output": "すでに正しく閉じられている括弧を消していくことで、まだ閉じられていない括弧を明らかにしていきます。\n\n1. < { } > を消すと [ < [ { [ ]  になる\n2. [ ] を消すと  [ < [ { になる\n3. [ < [ { を正しく閉じるには } ] > ] を追加する必要がある\n4. 元の配列 [ < < { } > [ { [ ]  に } ] > ] を追加する\n\nよって完成された配列は [ < < { } > [ { [ ] } ] > ] です。"
        },
        {
            "input": "以下の文はある数式を自然言語で表したものです。元の数式を四則演算の数式として表してください。\n1を3で割った結果に33を足して、210を掛ける。最後に100で割る。",
            "output": "ステップバイステップに数式を完成させていきます。\n\n1. 「1を3で割った結果に33を足す」は 1/3+33 とできます\n2. これに210をかけるには、直前が足し算なので全体を括弧で囲う必要があり、 (1/3+33)*210 になります\n3. 最後に100で割るには、直前が掛け算なのでそのまま (1/3+33)*210/100 となります\n\nよって元の数式を四則演算の数式として表すと (1/3+33)*210/100 になります。"
        },
        {
            "input": "ネットスラングでは、似た音を持つ別の短い表現に表されることがあります。例えばサーバーは鯖（さば）と表現されます。\n「消防 」というネットスラングはどのような意味を持つでしょうか？",
            "output": "「消防」の読みは「しょうぼう」で、この読みを持つ別の単語を考えると「小坊」などが考えられます。\n「小」は「小学生」「小さい」、「坊」は「子供」などの意味を持つことから、小学生・子供といった意味であることが考えられます。"
        },
        {
            "input": "今日の夕方、東京タワーに行くことはできる？\nという問いに答えるのに十分な情報はどれですか？\n\n情報1: 今日は3/1(金)で、明日は祝日だ。\n情報2: 東京タワーの営業状況について: 新型コロナウイルス感染拡大防止の為、営業時間を変更致します。 平日は21時まで、祝日は19時までの短縮営業となります。\n\n選択肢:\n- 情報1のみで十分で、情報2のみでは不十分\n- 情報2のみで十分で、情報1のみでは不十分\n- 情報1と2の両方で十分\n- 情報1, 2のどちらかのみで十分\n- 情報1, 2のどちらでも不十分",
            "output": "情報1で今日の日付について述べられていますが、情報2から東京タワーは平日・祝日問わず夕方の時間帯は営業していることがわかります。\n\nよって与えられた問いに答えるには「情報2のみで十分で、情報1のみでは不十分」です。"
        },
        {
            "input": "以下の文章から、話者の推測や感想を除いた客観的な事実のみを抽出してください。\n\n今日は雨だから遠足は中止になるだろうな〜 お菓子を買ったから皆と食べようと思ってたのに残念",
            "output": "与えられた文から客観的な事実を抽出すると、以下のようになります。\n\n- 今日は雨である\n- 話者はお菓子を買っていた\n\nまた文から「遠足が予定されていた」ことや「話者と皆で遠足に行く予定だった」ことなども示唆されています。"
        }
    ],
}

In [ ]:
def determine_task_type(input_text):
    """
    入力テキストに基づいてタスク種別を判定する。
    キーワードベースの判定を行い、閾値を超える場合はそのタスク種別と判定する。
    閾値を超えるタスク種別が複数ある場合は、「その他」と判定する。
    """
    scores = {}
    for task_type, keywords in KEYWORDS.items():
        score = 0
        for keyword, weight in keywords:
            if keyword in input_text:
                score += weight
        scores[task_type] = score

    # 閾値を設定
    threshold = 0.8

    # スコアが高い順にソート
    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)

    # 閾値を超えるタスク種別を取得
    top_task_types = [task_type for task_type, score in sorted_scores if score >= threshold]

    # 閾値を超えるタスク種別が複数ある場合は、「その他」と判定する
    if len(top_task_types) > 1:
        determined_task_type = 'その他'
    elif len(top_task_types) == 1:
        determined_task_type = top_task_types[0]
    else:
        determined_task_type = 'その他'  # 閾値を超えるタスク種別がない場合も「その他」とする

    params = PARAMS.get(determined_task_type, PARAMS['その他'])
    return determined_task_type, params

In [ ]:
def create_prompt(input_text):
    """タスク種別に応じたプロンプトの生成"""
    if not input_text:
        return "エラー: 入力テキストが空です。", "エラー", {}

    task_type, params = determine_task_type(input_text)
    instruction = INSTRUCTIONS.get(task_type, "以下の内容に基づいて、簡潔に回答してください。")

    # Few-shotの例を追加
    few_shot_examples = ""
    if task_type in FEW_SHOT_EXAMPLES:
        for example in FEW_SHOT_EXAMPLES[task_type]:
            few_shot_examples += f"### 入力:\n{example['input']}\n\n### 回答:\n{example['output']}\n\n"

    prompt = f"### 指示:\n以下の指示に従って、『簡潔』で分かりやすい回答を心がけて回答してください。\n{instruction}\n\n{few_shot_examples}### 入力:\n{input_text}\n\n### 回答:\n"

    # エラーハンドリング
    if task_type == "エラー":
        print(prompt)  # エラーメッセージを出力

    return prompt, task_type, params

In [ ]:
def create_prompts(data_list):
    """データセット全体に対してプロンプトを生成"""
    prompts = []
    task_types = []
    params_list = []
    for data in data_list:
        input_text = data['input']
        prompt, task_type, params = create_prompt(input_text)
        prompts.append(prompt)
        task_types.append(task_type)
        params_list.append(params)
    return prompts, task_types, params_list

In [ ]:
# グローバル変数としてTokenizerを一度だけ初期化
tokenizer = Tokenizer()

def validate_output(output: str, task_type: str) -> bool:
    """出力のバリデーションを行う関数"""
    if not basic_validation(output):
        return False
    if not quality_validation(output):
        return False
    if not instruction_validation(output):
        return False
    if not detect_unnecessary_phrases(output):
        return False
    return True

def basic_validation(text: str) -> bool:
    """基本的なバリデーションチェック"""
    if not text or text.isspace():
        print("出力が空またはスペースのみです")
        return False
    
    if len(text) < 10:
        print("出力が短すぎます")
        return False
    if len(text) > 2000:
        print("出力が長すぎます")
        return False
    
    if has_excessive_repetition(text):
        print("過度な文字列の繰り返しが検出されました")
        return False
    
    return True

def quality_validation(text: str) -> bool:
    """文章の品質チェック"""
    if not validate_readability(text):
        print("読みやすさに問題があります")
        return False
    
    return True

def instruction_validation(text: str) -> bool:
    """指示に沿った出力がされているかチェック"""
    if "### 回答:" in text:
        print("出力に'### 回答:'が含まれています")
        return False
    
    if "### 入力:" in text:
        print("出力に'### 入力:'が含まれています")
        return False
    
    return True

def detect_unnecessary_phrases(text: str) -> bool:
    """不要な文言が含まれていないかチェックする関数"""
    unnecessary_phrases = [
        r"再確認:.*",
        r"###",
        r"入力:.*",
        r"応答:.*",
        r"応答例:.*",
        r"評価基準:.*",
        r"指示:.*",
        r"指示が不明瞭な場合は.*"
        r"指示が不明確な場合は.*"
    ]
    for phrase in unnecessary_phrases:
        if re.search(phrase, text, re.DOTALL):
            print(f"不要な文言「{phrase}」が検出されました")
            return False
    return True

def has_excessive_repetition(text: str) -> bool:
    """過度な繰り返しのチェック（最適化版）"""
    # 文字数の閾値を事前に計算
    text_len = len(text)
    char_threshold = text_len * 0.3
    
    # 文字の出現回数をカウント（頻出文字のみ）
    char_count = {}
    for char in text:
        # すでに閾値を超えている文字があれば即座にTrue
        if char in char_count:
            char_count[char] += 1
            if char_count[char] > char_threshold:
                return True
        else:
            char_count[char] = 1
    
    # 単語の繰り返しチェックは、明らかな繰り返しパターンがある場合のみ実行
    if len(set(text)) / text_len < 0.2:  # 文字の種類が少ない場合のみ
        words = [token.surface for token in tokenizer.tokenize(text)]
        words_len = len(words)
        word_threshold = words_len * 0.2
        
        word_count = {}
        for word in words:
            if len(word) > 1:  # 2文字以上の単語のみチェック
                if word in word_count:
                    word_count[word] += 1
                    if word_count[word] > word_threshold:
                        return True
                else:
                    word_count[word] = 1
    
    return False

def validate_readability(text: str) -> bool:
    """読みやすさのチェック"""
    # 文の長さチェック
    sentences = re.split('[。！？!?]', text)
    sentences = [s for s in sentences if s.strip()]
    
    for sent in sentences:
        if len(sent) > 100:
            return False
    
    # 漢字の割合チェック
    kanji_ratio = len([c for c in text if 0x4E00 <= ord(c) <= 0x9FFF]) / len(text)
    if kanji_ratio > 0.6:
        return False
    
    return True


In [ ]:
class TaskCategory(Enum):
    FACTUAL = ['説明', '推論・分析', '事実確認']
    CREATIVE = ['生成・作成', 'ロールプレイ']
    ANALYTICAL = ['比較', '分類・カテゴライズ']
    STRUCTURED = ['ソート・並び替え']

@dataclass
class ParamConfig:
    """パラメータ設定のデータクラス"""
    base_temp: float
    base_top_p: float
    base_top_k: int
    base_repeat_penalty: float
    
    def get_decay_rate(self, retry_count: int) -> float:
        """リトライ回数に応じた減衰率を計算"""
        return np.exp(-0.1 * retry_count)

def adjust_generation_params(
    params: Dict[str, Any], 
    retry_count: int, 
    task_type: str,
    success_history: list = None
) -> Dict[str, Any]:
    """リトライ回数に応じて生成パラメータを最適化する関数"""
    
    # 基本設定
    config = ParamConfig(
        base_temp=0.7,
        base_top_p=0.9,
        base_top_k=40,
        base_repeat_penalty=1.3
    )
    
    # 元のパラメータをコピー
    adjusted_params = params.copy()
    
    # タスクカテゴリの判定
    task_category = None
    for category in TaskCategory:
        if task_type in category.value:
            task_category = category
            break
    
    # 成功率に基づく調整パラメータの計算
    if success_history:
        success_rate = sum(success_history) / len(success_history)
        # 成功率を-1から1の範囲にマッピング
        mu = (success_rate - 0.5) * 2
        sigma = 1.0
    else:
        mu = 0.0
        sigma = 1.0
        
    # リトライ回数に応じた基本調整
    decay_rate = config.get_decay_rate(retry_count)
    
    if retry_count == 1:
        # 1回目: 簡潔さと明瞭さを重視したベースライン設定
        adjusted_params.update({
            "temperature": config.base_temp,
            "top_p": config.base_top_p,
            "top_k": config.base_top_k,
            "repeat_penalty": config.base_repeat_penalty
        })
    elif retry_count == 2:
        # 2回目: タスクタイプに応じた調整
        if task_category == TaskCategory.FACTUAL:
            # 事実重視の設定
            adjusted_params.update({
                "temperature": max(config.base_temp - 0.2, 0.1),
                "top_p": max(config.base_top_p - 0.1, 0.6),
                "top_k": max(config.base_top_k - 10, 20),
                "repeat_penalty": min(config.base_repeat_penalty + 0.2, 2.0)
            })
        elif task_category == TaskCategory.CREATIVE:
            # 創造性許容の設定
            adjusted_params.update({
                "temperature": min(config.base_temp + 0.2, 1.0),
                "top_p": config.base_top_p,
                "top_k": config.base_top_k,
                "repeat_penalty": max(config.base_repeat_penalty - 0.2, 1.0)
            })
        else:
            # その他のタスクはバランス重視
            adjusted_params.update({
                "temperature": config.base_temp,
                "top_p": config.base_top_p,
                "top_k": config.base_top_k,
                "repeat_penalty": config.base_repeat_penalty
            })
    elif retry_count == 3:
        # 3回目: 成功率に基づいた調整
        if success_history and success_rate < 0.5:
            # 成功率が低い場合はより保守的な設定
            adjusted_params.update({
                "temperature": max(config.base_temp - 0.3, 0.1),
                "top_p": max(config.base_top_p - 0.2, 0.5),
                "top_k": max(config.base_top_k - 20, 10),
                "repeat_penalty": min(config.base_repeat_penalty + 0.3, 2.5)
            })
        else:
            # 成功率が高い場合は探索を広げる
            adjusted_params.update({
                "temperature": min(config.base_temp + 0.1, 1.0),
                "top_p": min(config.base_top_p + 0.1, 1.0),
                "top_k": min(config.base_top_k + 10, 60),
                "repeat_penalty": max(config.base_repeat_penalty - 0.1, 1.0)
            })
    elif retry_count == 4:
        # 4回目: 減衰率に基づいた微調整
        adjusted_params.update({
            "temperature": max(config.base_temp - 0.2 * decay_rate, 0.1),
            "top_p": max(config.base_top_p - 0.1 * decay_rate, 0.6),
            "top_k": max(int(config.base_top_k * (1 - 0.1 * decay_rate)), 20),
            "repeat_penalty": min(config.base_repeat_penalty + 0.2 * decay_rate, 2.0)
        })
    elif retry_count == 5:
        # 5回目: 減衰率に基づいた微調整
        adjusted_params.update({
            "temperature": max(config.base_temp - 0.25 * decay_rate, 0.1),
            "top_p": max(config.base_top_p - 0.15 * decay_rate, 0.5),
            "top_k": max(int(config.base_top_k * (1 - 0.15 * decay_rate)), 15),
            "repeat_penalty": min(config.base_repeat_penalty + 0.25 * decay_rate, 2.2)
        })
    elif retry_count == 6:
        # 6回目: 減衰率に基づいた微調整
        adjusted_params.update({
            "temperature": max(config.base_temp - 0.3 * decay_rate, 0.1),
            "top_p": max(config.base_top_p - 0.2 * decay_rate, 0.4),
            "top_k": max(int(config.base_top_k * (1 - 0.2 * decay_rate)), 10),
            "repeat_penalty": min(config.base_repeat_penalty + 0.3 * decay_rate, 2.5)
        })
    elif retry_count == 7:
        # 7回目: 減衰率に基づいた微調整
        adjusted_params.update({
            "temperature": max(config.base_temp - 0.35 * decay_rate, 0.1),
            "top_p": max(config.base_top_p - 0.25 * decay_rate, 0.3),
            "top_k": max(int(config.base_top_k * (1 - 0.25 * decay_rate)), 5),
            "repeat_penalty": min(config.base_repeat_penalty + 0.35 * decay_rate, 3.0)
        })
    elif retry_count == 8:
        # 8回目: タスク種別に応じた最終調整
        if task_category == TaskCategory.FACTUAL:
            adjusted_params.update({
                "temperature": 0.1,
                "top_p": 0.5,
                "top_k": 10,
                "repeat_penalty": 2.5,
                "presence_penalty": 1.5,
                "frequency_penalty": 1.5
            })
        elif task_category == TaskCategory.CREATIVE:
            adjusted_params.update({
                "temperature": 1.0,
                "top_p": 0.95,
                "top_k": 60,
                "repeat_penalty": 1.0,
                "presence_penalty": 0.5,
                "frequency_penalty": 0.5
            })
        else:
            # その他のタスクは保守的な設定
            adjusted_params.update({
                "temperature": 0.2,
                "top_p": 0.6,
                "top_k": 20,
                "repeat_penalty": 2.0,
                "presence_penalty": 1.0,
                "frequency_penalty": 1.0
            })
    
    # パラメータの範囲を確認・調整
    adjusted_params["temperature"] = np.clip(adjusted_params["temperature"], 0.1, 1.0)
    adjusted_params["top_p"] = np.clip(adjusted_params["top_p"], 0.1, 1.0)
    adjusted_params["top_k"] = max(1, int(adjusted_params["top_k"]))
    
    logging.info(f"Adjusted parameters for retry {retry_count}: {adjusted_params}")
    
    return adjusted_params

In [ ]:
def generate_response(model, prompt, generation_params, task_type, task_id):
    max_retries = 8
    retries = 0
    success_history = []  # 成功履歴を追加

    while retries < max_retries:
        try:
            current_params = adjust_generation_params(
                generation_params, 
                retries + 1, 
                task_type,
                success_history
            )
            
            response = model.create_completion(
                prompt=prompt,
                **current_params
            )
            output = response["choices"][0]["text"].strip()

            if validate_output(output, task_type):
                success_history.append(1)
                return output
            else:
                success_history.append(0)
                logger.info(f"バリデーションエラー: タスクID {task_id}, リトライ {retries + 1} 回目")
                retries += 1

                # プロンプトに指示を追加
                if retries == 1:
                    # 簡潔で分かりやすい回答を強調
                    prompt += "\n\n**注意:** 回答は簡潔かつ要点を絞って記述してください。冗長さは避け、明確で理解しやすい説明を心がけてください。"
                elif retries == 2:
                    # 具体例やステップバイステップでの説明を必要に応じて追加
                    prompt += "\n\n**改善のためのヒント:** 必要であれば、具体的な例を挙げて説明してください。複雑な問題の場合は、ステップバイステップで段階的に説明すると分かりやすくなります。"
                elif retries == 3:
                    # 指示に厳密に従うように強調
                    prompt += "\n\n**再確認:** 最初の指示に厳密に従って回答してください。特に「簡潔」で「分かりやすい」回答を心がけてください。"
                elif retries == 4:
                    # 最終確認と警告
                    prompt += "\n\n**最終確認:** これが最後のチャンスです。これまでの指示をすべて考慮し、最も適切で簡潔な回答を作成してください。不明瞭な点や冗長さがないか、最終確認を怠らないでください。"

        except Exception as e:
            logger.error(f"推論エラー: タスクID {task_id}, リトライ {retries + 1} 回目, エラー: {str(e)}")
            retries += 1

    logger.error(f"最大リトライ回数を超えました: タスクID {task_id}")
    return ''

In [ ]:
def main():
    """メイン処理"""
    # モデルの読み込み
    llm = Llama(
        model_path=MODEL_PATH,
        lora_path=lora_path,
        n_ctx=4096,          # コンテキストサイズを調整
        n_gpu_layers=-1,     # GPU層を増やす
        verbose=True         # 詳細なログを有効化
    )

    # テストデータの読み込み
    with open('elyza-tasks-100-TV_0.jsonl', 'r', encoding='utf-8') as f:
        test_data = [json.loads(line) for line in f]

    # プロンプトの事前生成
    prompts, task_types, params_list = create_prompts(test_data)

    print(prompts[0])

    # 推論処理
    results = []
    for prompt, task_type, params, data in tqdm(zip(prompts, task_types, params_list, test_data), desc="推論処理中", total=len(test_data)):
        input_text = data['input']
        task_id = data['task_id']

        # generate_response 関数を呼び出して推論を実行
        output = generate_response(llm, prompt, params, task_type, task_id)

        results.append({
            "task_id": task_id,
            'input': input_text,
            'output': output,
            'task_type': task_type,
            'generation_params': {
                'max_tokens': params.get('max_tokens'),
                'temperature': params.get('temperature')
            }
        })

    # 結果の保存
    with open(FILE_NAME, 'w', encoding='utf-8') as f:
        for result in results:
            json.dump(result, f, ensure_ascii=False)
            f.write('\n')

In [ ]:
if __name__ == "__main__":
    main()